## 处理（合并）新的公司数据

In [2]:
import pandas as pd

heilishi_path = '/data/data-lakes/ssc/helishi_new.xlsx'

### 分别载入数据

In [9]:
helishi_dfs = pd.read_excel(heilishi_path)
helishi_dfs = helishi_dfs[['技能', '一级分类', '标准问(必填)', '答案(最多10条)','扩展问(最多1000条)']]
helishi_rename = helishi_dfs.rename(columns={'技能': 'skill', '一级分类': 'category1',
    '标准问(必填)': 'question', '扩展问(最多1000条)': 'alias', '答案(最多10条)':'answer'})

helishi_rename.head()

,skill,category1,question,answer,alias
0,公司概况,公司简介,贺利氏是一家什么样的集团？,您好，总部位于德国哈瑙市的贺利氏是一家全球领先的科技集团。公司在1660年从一间小药房起家，...,贺利氏集团是一家怎样的公司？
1,NaN,NaN,NaN,NaN,贺利氏集团是干嘛的？
2,NaN,NaN,NaN,NaN,贺利氏集团是一家做什么的公司？
3,NaN,NaN,NaN,NaN,贺利氏集团是做什么的？
4,NaN,NaN,NaN,NaN,贺利氏集团是干什么的？


### 填充数据缺失的部分

In [12]:
# 设置提醒
# pd.options.mode.chained_assignment = None
helishi_ffill = helishi_rename[['skill', 'category1', 'question']].fillna(method='ffill', inplace=False)
helishi_ffill.loc[:, 'alias'] = helishi_rename.loc[:, 'alias']
helishi_ffill.head()

,skill,category1,question,alias
0,公司概况,公司简介,贺利氏是一家什么样的集团？,贺利氏集团是一家怎样的公司？
1,公司概况,公司简介,贺利氏是一家什么样的集团？,贺利氏集团是干嘛的？
2,公司概况,公司简介,贺利氏是一家什么样的集团？,贺利氏集团是一家做什么的公司？
3,公司概况,公司简介,贺利氏是一家什么样的集团？,贺利氏集团是做什么的？
4,公司概况,公司简介,贺利氏是一家什么样的集团？,贺利氏集团是干什么的？


### 验证先前的数据格式

In [14]:
previous_path = '/data/projects/negative-sample-with-KMeans/data/faq_corpus_with_index.xlsx'

dfs_previous = pd.read_excel(previous_path)
common_columns = ['skill', 'category1', 'question', 'alias']
common_columns

['skill', 'category1', 'question', 'alias']

### 合并数据并去重

In [26]:
dfs_previous_common = dfs_previous[common_columns]
helishi_ffill_common = helishi_ffill[common_columns]

dfs_merge = pd.concat([dfs_previous_common, helishi_ffill_common], ignore_index=True)

dfs_filter = dfs_merge.drop_duplicates(subset='alias', keep='first',ignore_index=True)
dfs_filter.head()

,skill,category1,question,alias
0,公司概述,公司介绍,公司的投资方是哪家,公司投资方是谁
1,公司概述,公司介绍,公司的投资方是哪家,谁是公司投资方
2,公司概述,公司介绍,公司的投资方是哪家,谁给公司融资的
3,公司概述,公司介绍,公司的投资方是哪家,投资方是谁
4,公司概述,公司介绍,公司的投资方是哪家,投资方有谁


### 构造ID映射关系

In [27]:
# skill, category1, question
column_maps = ['skill', 'category1', 'question']

id_mapping = {}
for column in column_maps:
    items = dfs_filter[column].dropna().drop_duplicates().tolist()
    id_mapping[column] = dict(zip(items, range(1, len(items) + 1)))
    dfs_filter.loc[:, column + '_' + 'id'] = dfs_filter.loc[:, column].apply(lambda x: id_mapping[column][x])

dfs_filter.loc[:, 'index'] = dfs_filter.index + 1
dfs_filter.head()

/data/opts/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/data/opts/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,skill,category1,question,alias,skill_id,category1_id,question_id,index
0,公司概述,公司介绍,公司的投资方是哪家,公司投资方是谁,1,1,1,1
1,公司概述,公司介绍,公司的投资方是哪家,谁是公司投资方,1,1,1,2
2,公司概述,公司介绍,公司的投资方是哪家,谁给公司融资的,1,1,1,3
3,公司概述,公司介绍,公司的投资方是哪家,投资方是谁,1,1,1,4
4,公司概述,公司介绍,公司的投资方是哪家,投资方有谁,1,1,1,5
